## Compute tick count using dedicated service(C#)

### Overview
Tick count can be used as an indicator to illustrate market activity by computing the technical transactions/transmissions count on a tick by tick basis.

It represents a valuable technical indicator to monitor tick by tick data reception. Building a history of tick count can be useful in the following use-cases:
* guarantee tick by tick data good reception and completion
* compare trading periods and highlight high-level of activity periods
* combined with other indicators, it helps to optimal future rolling 
* etc.

Correlated with a set of other indicators such as traded volume and count of trades, over the same look back period, it repserents a good measurement of the market quality.

This samples enables to retrieve **on-demand** tick count data points by calling a dedicated service.

### Inputs/outputs
Tick count sample requires instrument's identifier, date time intervals and time granularity as per inputs. It returns the tick count sampled using the input time granularity.

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

<div class="alert alert-block alert-info">
    <b>Note:</b> A similar scenario dedicated to future contract's tick count, all maturities covered, is available in <a href="https://ganymede.systemathics.com/user/talel.nasri@systemathics.com/lab/tree/csharp/future_roll.ipynb">future roll strategy</a> notebook.
</div>

### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* *TopologiesService*: to directly retrieve ticks objects from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Topology.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run tick count sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Topology.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Topology parmeters

In [ ]:
var granularity = TopologyGranularity.Daily;
var level = Level.TradesAndBook;

#### 3.2 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate the topologies request
var request = new TopologiesRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Granularity = granularity ,
    Level = level
    };

// Instantiate the topologies service
var service = new TopologiesService.TopologiesServiceClient(channel);

### Step4: Visualize data

#### 4.1 Retrieve tick count data

In [ ]:
// Process the topologies request
var reply = service.Topologies(request, headers);

// Generate request response and total tick count
var topologies = reply.Entries;
var totalTicks = topologies.Sum(t=> (double)t.TicksCount);

// Retrieve daily tick count
var topologiesSelection = topologies.Select(t => new Tuple<DateTime, double>(new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount));

#### 4.2 Plot tick count
Plot daily tick count over the look back period from the request result and plot them with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [ ]:
var chart = Chart.Column(topologiesSelection);
chart.WithTitle($"{granularity} ticks count for {ticker} - Total ticks count: {totalTicks:N0}");
chart.WithYTitle("#Ticks");
chart.WithXTitle("Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);